$\newcommand{\To}{\Rightarrow}$

In [2]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [19]:
from kernel.type import TVar, TFun
from kernel.term import Term, Var
from logic import nat
from logic.nat import natT, zero, one, plus
from logic import basic
from logic import logic
from syntax import printer

thy = basic.loadTheory('nat')

## Substitution

Substitution on terms is analogous to substitution on types, but more complicated due to the presence of abstractions (lambda terms), and the need to substitute for both type and term variables. Consider the following term:

In [25]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
t = Term.mk_equals(a, b)
print(repr(t))

Comb(Comb(Const(equals,'a => 'a => bool),Var(a,'a)),Var(b,'a))


Observe that there are both type and term variables in $t$. The method `subst_type` takes a dictionary of assignments for type variables, and substitutes using this dictionary.

In [11]:
t2 = t.subst_type({"a": natT})
print(repr(t2))

Comb(Comb(Const(equals,nat => nat => bool),Var(a,nat)),Var(b,nat))


Next, we can apply the method `subst`, which takes a dictionary of assignments for (term) variables, and substitutes using this dictionary.

In [19]:
t3 = t2.subst({"a": zero, "b": one})
print(printer.print_term(thy, t3))

0 = 1


We now look at some examples demonstrating how substitution interacts with abstractions. Consider the following term $t$:

In [5]:
x = Var("x", natT)
y = Var("y", natT)
t = plus(Term.mk_abs(x, plus(x, y))(one), x)
print(printer.print_term(thy, t, unicode=True))

(λx. x + y) 1 + x


This term contains variables $x$ and $y$. Let's substitute 3 for $x$ and 5 for $y$:

In [9]:
t2 = t.subst({"x": nat.to_binary(3), "y": nat.to_binary(5)})
print(printer.print_term(thy, t2, unicode=True))

(λx. x + 5) 1 + 3


Observe that both the $x$ at the end and the $y$ inside the lambda term is substituted. However, the $x$ inside the lambda term is not. This is because the latter $x$ is a bound variable, and quite different from the former $x$. In fact, since the name of the bound variable does not matter, the term $t$ is equivalent to $(\lambda z. z + y) 1 + x$. The fact that the name of the bound variable is the same as a variable outside the lambda term is simply a coincidence.

By default, substitution does not perform $\beta$-conversion (evaluation of functions). For example:

In [18]:
two = nat.to_binary(2)
f = Var("f", TFun(natT, natT))
a = Var("a", natT)
t = f(a)
print(printer.print_term(thy, t))

t2 = t.subst({"f": Term.mk_abs(x, plus(x,two)), "a": two})
print(printer.print_term(thy, t2, unicode=True))

f a
(λx. x + 2) 2


To evaluate $f$ after a substitution, one can use `logic.beta_norm` function introduced previously. The function `logic.subst_norm` combines the three operations above: type substitution, term substitution, and $\beta$-normalization. This function takes the pattern to be substituted, and a pair of dictionaries of assignments for type and term variables.

In [24]:
t3 = logic.subst_norm(t, (dict(), {"f": Term.mk_abs(x, plus(x,two)), "a": two}))
print(printer.print_term(thy, t3))

2 + 2


We can also use `logic.subst_norm` to do the example at the beginning of the section:

In [29]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
t = Term.mk_equals(a, b)
t2 = logic.subst_norm(t, ({"a": natT}, {"a": zero, "b": one}))
print(printer.print_term(thy, t2))

0 = 1


## Matching